In [1]:
import requests
from bs4 import BeautifulSoup
import time
import urllib.request 
from selenium.webdriver import Chrome
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,ElementClickInterceptedException,ElementNotInteractableException
import datetime as dt
from urllib.request import urlopen
import pandas as pd

## 네이버 VIEW Crawling

In [ ]:
start_url = 'https://www.naver.com'
browser.get(start_url)
browser.maximize_window()

In [3]:
jeonju_dong = pd.read_csv('전주.csv')['행정동'].to_list()

In [5]:
def NaverView_Crawling(dong) :
    
    browser.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/div/div[3]/form/fieldset/button').click() ### 검색결과로 가기
    time.sleep(1.5)
    
    browser.find_element_by_xpath('/html/body/div[3]/div[1]/div/div[2]/div[1]/div/ul/li[2]/a').click() ### VIEW 클릭
    time.sleep(1.2)
    
    browser.find_element_by_xpath('/html/body/div[3]/div[1]/div/div[1]/div[1]/div/form/fieldset/div[1]/input').send_keys(dong)
    time.sleep(1.3)
    
    browser.find_element_by_xpath('/html/body/div[3]/div[1]/div/div[1]/div[1]/div/form/fieldset/button').click() ### 동 검색 클릭
    time.sleep(1.5)
    
    #### 끝까지 내리기
    
    prev_height = browser.execute_script("return document.body.scrollHeight")
    
    while True :
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(1)
    
        curr_height = browser.execute_script("return document.body.scrollHeight")
    
        if curr_height == prev_height :
            break
        else :
            prev_height = browser.execute_script("return document.body.scrollHeight")
            
    view_name = []        
    date_list = []
    view_list = []
    
    for i in range(1,100000) :
        try :
            view_name.append(browser.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/section/div/div[2]/panel-list/div/more-contents/div/ul/li[{}]/div/div[1]/span/span/span[2]/a'.format(i)).text) ### view명
            date_list.append(browser.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/section/div/div[2]/panel-list/div/more-contents/div/ul/li[{}]/div[1]/div[1]/span/span/span[1]/span'.format(i)).text) ### 날짜
            try :
                view_list.append(browser.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/section/div/div[2]/panel-list/div/more-contents/div/ul/li[{}]/div[1]/a[2]'.format(i)).text) ### 블로그    
            except NoSuchElementException :
                view_list.append(browser.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/section/div/div[2]/panel-list/div/more-contents/div/ul/li[{}]/div/a'.format(i)).text) ##카페
                
        except NoSuchElementException :
            break
            
    naver_df = pd.DataFrame(columns=['행정동'])        
    naver_df['뷰_이름'] = view_name
    naver_df['날짜'] = date_list
    naver_df['제목'] = view_list
    naver_df['행정동'] = dong
    
    browser.find_element_by_xpath('/html/body/div[3]/div[1]/div/div[1]/div[1]/div/h1/a/img').click() ## 메인화면으로 돌아가기
    time.sleep(1.5)
    
    return naver_df

In [6]:
naver_jeonju = pd.DataFrame(columns=['행정동','뷰_이름','날짜','제목'])
for dong in jeonju_dong :
    naver_jeonju = pd.concat([naver_jeonju,NaverView_Crawling(dong)],axis = 0)

In [9]:
naver_jeonju =naver_jeonju.reset_index(drop = True)

In [10]:
naver_jeonju.to_csv("전주_네이버.csv",index = False)

In [11]:
naver_jeonju

,행정동,뷰_이름,날짜,제목
0,금암1동,더런드리 셀프빨래방 전주금암점,2021.03.16.,금암 1동에서 금암 2동으로 이사하신 고객님께서 이불 빨래하셨어요.
1,금암1동,다큐TV,2020.07.15.,"[다큐TV]전주시, 금암1동 정체성 담긴 마을로고 공모"
2,금암1동,"전주부동산의 모든것 카페-효천지구,만성지구,에코시티,혁신도시",2021.04.02.,금암1동 단톡주택단지 내놓습니다 (교보생명 뒤)
3,금암1동,전주 뚱땡이 샌드위치,2020.07.14.,전주시청*금암1동주민*우아1동주민*잇올스파르타 독서실^^
4,금암1동,◆피터팬의 좋은방구하기◆전주시점-원룸/오피스텔/부동산직거래,2020.09.28.,금암1동쪽 월세집구합니다.
...,...,...,...,...
3342,효자5동,"우마동- 강아지,고양이무료분양,애완동물,고슴도치,열대어분양",2021.02.28.,❤️홈딜리버리❤️오랜만에 돌아온 미유펫인천점의 병원오픈기념❤️ 특별할인이벤트!!사랑...
3343,효자5동,"우마동- 강아지,고양이무료분양,애완동물,고슴도치,열대어분양",6일 전,전국홈딜리버리미유펫인천점말티55~비숑55~푸들65~!1층분양/미용 2층자체병원인천최...
3344,효자5동,"강아지분양,고양이분양,강아지,고양이,유기견묘,무료분양 미유펫",2020.10.16.,부산강아지분양 100여마리의 아기들을! 모두다 할인이벤트가격으로 분양중입니당♥(새로...
3345,효자5동,"우마동- 강아지,고양이무료분양,애완동물,고슴도치,열대어분양",2021.04.25.,전국홈딜리버리미유펫인천점말티55~비숑55~푸들65~!1층분양/미용 2층자체병원인천최...


# Instagram

In [3]:
def Insta_Crawling(want,df_name,target_num) :
    
    delay = 2.0
    browser = Chrome()
    
    start_url = 'https://www.instagram.com/?hl=ko'
    browser.get(start_url)
    browser.maximize_window()
    time.sleep(1)
    browser.find_element_by_xpath('/html/body/div[1]/section/main/article/div[2]/div[1]/div/form/div/div[1]/div/label/input').send_keys('01053063058')
    time.sleep(1)
    browser.find_element_by_xpath('/html/body/div[1]/section/main/article/div[2]/div[1]/div/form/div/div[2]/div/label/input').send_keys('!@Whdid501')
    time.sleep(1)
    browser.find_element_by_xpath('/html/body/div[1]/section/main/article/div[2]/div[1]/div/form/div/div[3]').click()
    time.sleep(7)
    browser.find_element_by_xpath('/html/body/div[1]/section/main/div/div/div/div/button').click()
    time.sleep(1)
    browser.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()
    time.sleep(2)
    
    browser.get('https://www.instagram.com/explore/tags/{}'.format(want))
    time.sleep(10)
    
    first = browser.find_element_by_css_selector('div._9AhH0')
    time.sleep(2)
    first.click()
    time.sleep(5)
    hashtags = []
    
    for i in range(target_num) :
        if browser.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]').text =='' or browser.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/header/div[2]/div[1]/div[1]/span/a').text not in browser.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul').text.split('\n'):
            hashtags.append(' ')
            time.sleep(4.5)
        
        elif browser.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]').text !='' :
            id_list = []
            for i in range(1,100) :
                try :
                    id_list.append(browser.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul/ul[{}]/div/li/div/div[1]/div[2]/h3/div/span/a'.format(i)).text)
                except NoSuchElementException :
                    break
        
            time.sleep(4.5)
            number_list = [i+1 for i,v in enumerate(id_list) if v == browser.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/header/div[2]/div[1]/div[1]/span/a').text]
        
            for num in number_list :
                if '답글 보기' in browser.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul/ul[{}]'.format(num)).text:
                    browser.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul/ul[{}]/li/ul/li/div/button/span'.format(num)).click()    
        
            value = []
            for t in re.findall(r'#[^\s#,\\]+',browser.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul').text) :
                value.append(t.replace('#',''))
            
            hashtags.append(value)
            time.sleep(4.5)
        
        elif '#' in browser.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul').text :
            value = []
            for t in re.findall(r'#[^\s#,\\]+',browser.find_element_by_xpath('/html/body/div[45/div[2]/div/article/div[3]/div[1]/ul').text):
                value.append(t.replace('#',''))
        
            hashtags.append(value)
            time.sleep(4.5)
        
        else :
            hashtags.append(' ')
            time.sleep(4.5)
        try :
            right = browser.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow')
            right.click()
            time.sleep(5)
        except NoSuchElementException:
            break

    df = pd.DataFrame(columns=['hashtags'])
    df['hashtags'] = [' '.join(hashtags[i]) for i in range(len(hashtags))]
    df['키워드'] = str(df_name)
    df.to_csv("{}해시태그.csv".format(df_name),index = False)
    
    browser.find_element_by_xpath('/html/body/div[5]/div[3]/button').click()
    time.sleep(1.5)
    browser.close()
    time.sleep(3)
    
    return df.head()

In [106]:
Insta_Crawling(tag_list[0],'전주해시태그',500)

,hashtags
0,출근 daily dailylook instadaily goodday now fff ...
1,korea selfie fff f4f lfl ootd daily dailylook ...
2,전주 송천동 에코시티 세병공원 세병호 씽씽 03 19 열아홉 고3 날씨좋다☀️ 나들...
3,selfie selica selstagram daily ootd dailylook ...
4,충장로카페 팔로우 셀카 셀피 동성로 둔산동 가로수길 신시가지 수성못 일상 데일리 감...


In [5]:
tag_list = [ '전주여행', '전주카페','전주맛집','전주한옥마을','전주데이트', '전주신시가지', '전주가볼만한곳','전주핫플', '전주투어']

In [14]:
Insta_Crawling(tag_list[1],'전주카페',300)

,hashtags,키워드
0,coffee cafetour ondocamera cafestagram ondocam...,전주카페
1,제주 제주도 jeju insta 전주 전주근교카페 전주카...,전주카페
2,왓더블랙 왓더그레이 전주맛집추천 전북대맛집 전주객사맛집 객사맛집 객리단길맛집 전주신...,전주카페
3,빈타이 빈타이혁신점 전주카페 전주혁신도시 객리단길 객리단길카페 전주카페추천 카페추천...,전주카페
4,인스타그램 셀스타그램 얼스타그램 셀피 셀카 일상 소통 좋아요 좋아요반사 좋반 첫줄 ...,전주카페


In [17]:
Insta_Crawling(tag_list[0],'전주여행',300)

,hashtags,키워드
0,일상 데일리 일상스타그램 데일리그램 여행스타그램 전주여행 전주한옥마을 전주 여행에미...,전주여행
1,전주 전주여행 전주한옥마을 전라도여행 전주한옥마을한복 전주여행코스 애견동반전주여행 ...,전주여행
2,전주여행 조팝나무 공조팝 전주수목원 찍길동_전주 전주가볼만한곳 전주 전주핫플레이스 ...,전주여행
3,홍오커플 경기전 경기전돌담길 전주한옥마을 전주한옥마을맛집 전주카페 전주가볼만한곳 라...,전주여행
4,한복남 전주한옥마을 전주한옥마을한복대여 전주한옥마을한복대여는한복남 전주여행 한복 한...,전주여행


In [24]:
Insta_Crawling(tag_list[2],'전주맛집',300)

,hashtags,키워드
0,브런치 전주맛집 전주브런치 전주요거트 프렌치토스트 겹빵 신시가지카페 고하우스,전주맛집
1,전주맛집 객리단길 목련 전주 전주이자카야 전주맛집추천 전북대맛집 전주객사맛집 객사맛...,전주맛집
2,전주 방구석1열 크림파스타 육회비빔밥 전주맛집 맛스타그램 먹스타그램 맛스타 먹스타 ...,전주맛집
3,전주맛집 다가생구이 객사맛집 국내여행 전주가볼만한곳 먹방 셀카 셀피 얼스타그램 인친...,전주맛집
4,전주맛집 요가파이어 전주 신시가지 매운맛집 전주맛집추천 전북대맛집 전주객사맛집 객사...,전주맛집


In [30]:
Insta_Crawling(tag_list[3],'전주한옥마을',300)

,hashtags,키워드
0,전주경성의복 전주한옥마을한복대여 전주한복대여 전주한옥마을 전주한옥마을카페 전주한옥마...,전주한옥마을
1,전주맛집 전주 객리단길 리틀버터 전주맛집추천 전북대맛집 전주객사맛집 객사맛집 객리단...,전주한옥마을
2,전주맛집 전주 데이트코스 전주 실내데이트 전주맛집추천 전북대맛집 전주객사맛집 객사맛...,전주한옥마을
3,전주한옥마을 한복 추억팔이 추팔중 포토그래피 선팔 맞팔 고딩스타그램 고1 05년생 ...,전주한옥마을
4,한복남 전주한옥마을 전주한옥마을한복대여 전주한옥마을한복대여는한복남 전주여행 한복 한...,전주한옥마을


In [ ]:
for i in range(5,len(tag_list)) :
    Insta_Crawling(tag_list[i],tag_list[i],300)

## T-Map

In [5]:
search_df = pd.read_csv('검색지.csv')

In [8]:
import os

In [11]:
tmap_dir = os.getcwd() + "\\tmap19\\"
tmap = pd.DataFrame()
for f in os.listdir(tmap_dir) :
    tmap = pd.concat([tmap,pd.read_csv(tmap_dir + f)],axis = 0,ignore_index=True)

In [13]:
tmap =tmap.rename(columns={"지역(시도)" : "시도","지역(시군구)" : "시군구"})
jeonju_list = ['전주시 덕진구', '전주시 완산구']
tmap_jj = tmap.query('시군구 in @jeonju_list')
tmap_jj = tmap_jj.sort_values(by='일자').reset_index(drop = True)

In [15]:
search_list = ['여행/레저', '쇼핑','생활편의']
tmap_df = tmap_jj.query('검색지유형1 in @search_list')

In [32]:
search_19 = sorted(set(tmap_df['검색지명']))

In [34]:
search_19.extend(search_df['검색지'].tolist())

In [39]:
search_list = sorted(set(search_19))

In [48]:
start_url = 'https://map.kakao.com/'
browser.get(start_url)
browser.maximize_window()

In [64]:
search_list = ['전주 ' + value for value in search_list]

In [67]:
road_list = []
for i in range(len(search_list)) :
    browser.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/input').send_keys(search_list[i])
    time.sleep(1.5)
    
    browser.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/button').click()
    time.sleep(1.5)
    try :
        
        road = browser.find_element_by_css_selector('#info\.search\.place\.list > li:nth-child(1) > div.info_item > div.addr > p:nth-child(1)').text
        if road.split(' ')[0] != '전북' :
            road_list.append('no')
            time.sleep(1.5)
        else :
            road_list.append(road)
            time.sleep(1.5)
    except NoSuchElementException :
        road_list.append("NoExist")
        
    browser.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/input').click()
    browser.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/input').clear()
    time.sleep(1.5)

In [75]:
search_df = pd.DataFrame([search_list[i].split(' ')[1] for i in range(len(search_list))],columns=['검색지명'])

In [77]:
search_df['도로명주소'] = road_list

In [85]:
search_df = search_df.loc[search_df['도로명주소'] != 'NoExist'].reset_index(drop = True)

In [89]:
start_url = 'https://www.juso.go.kr/openIndexPage.do'
browser.get(start_url)
browser.maximize_window()

In [98]:
search_df['도로명주소'][36]

'전북 전주시 덕진구 인후동2가 237-3'

In [94]:
len(new_list)

36

In [ ]:
new_list = []
for ad in search_df['도로명주소'] :
    address = " ".join(ad.split(' ')[:5])
    browser.find_element_by_xpath('/html/body/div[3]/main/div/section[1]/div[1]/form/fieldset/div/div[1]/input[1]').send_keys(address)
    browser.find_element_by_xpath('/html/body/div[3]/main/div/section[1]/div[1]/form/fieldset/div/button').click()
    time.sleep(1.5)
    try :
        
        browser.find_element_by_css_selector('#list1 > div.addrWrap > div.addAddr').click()
        new_list.append(browser.find_element_by_xpath('/html/body/div[2]/div/main/div[3]/div[2]/ol/li/div[2]/ul/li[3]/span[1]').text)
        time.sleep(1.5)
    except NoSuchElementException :
        new_list.append("No")
    browser.back()
    time.sleep(1.5)

In [102]:
index_list = []
for i,v in enumerate(new_list) :
    if v =='No' :
        index_list.append(i)

In [117]:
search_df['행정동'] = new_list

In [119]:
dong_list = []
for i,v in enumerate(search_df['행정동']) :
    if v=='No' :
        dong_list.append(search_df['도로명주소'][i])
    else :
        dong_list.append(v.split(' ')[3])

In [121]:
search_df['행정동'] = dong_list

In [127]:
search_df['행정동'][index_list] = dongs

In [131]:
search_df['행정동'].unique()

array(['효자5동', '중앙동', '효자3동', '효자4동', '풍남동', '호성동', '덕진동', '동서학동', '조촌동',
       '삼천3동', '송천2동', '금암1동', '진북동', '노송동', '송천1동', '팔복동', '서신동', '우아2동',
       '혁신동', '인후2동', '인후3동', '서서학동', '중화산2동', '평화2동', '중화산1동', '효자2동',
       '여의동', '우아1동', '금암2동', '완산동'], dtype=object)

In [133]:
search_df.to_csv('tmapdong.csv',index = False)